In [2]:
#%%
from bs4 import BeautifulSoup
import requests
import numpy as np
#import requests as r
from tkinter import Label, Frame, Button, LabelFrame, Tk, StringVar
from tkinter.ttk import Notebook,Combobox,Treeview,Scrollbar
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import threading
import textwrap
from numpy import arange
from tkcalendar import DateEntry
import pandas as pd
from datetime import datetime,timedelta
from colors import red, green, black 

class CryptoMonitor():
    def __init__(self,master):
        self.master = master
        self.master.geometry("700x600")
        self.master.title('Crypto Monitor') 
        #self.master.resizable(0,0)
        self.Coin = StringVar(value="BTC-USD")
        self.precio = 0
        startDate = datetime.now()+timedelta(-30)

        endDate = datetime.now()
        Coins = ["Bitcoin","Ethereum","Bitcoin-Cash","Litecoin","Ethereum-Classic","Basic-Attention-Token","Zcash"]
        Coins2 = ["BTC-USD","ETH-USD","BCH-USD","LTC-USD","ETC-USD","BAT-USD","ZEC-USD"]
        Acronimos = ["BTC","ETH","BCH","LTC","ETC","BAT","ZEC"]
        self.Coins_dic = dict(zip(Coins2,Acronimos))
        self.Coins_dic2 = dict(zip(Coins2,Coins))
        
        frm = Frame(master)
        frm1 = LabelFrame(frm,text="Precio")
        frm2 = LabelFrame(frm, text="Rango de fecha")
        frm3 = LabelFrame(frm, text="Tabla y graficos")
        
        frm.pack()
        frm1.grid(column=0,row=0,padx=7,pady=5)
        frm2.grid(column=0,row=1)
        frm3.grid(column=0,row=2,padx=7,pady=7)
        
        fig = Figure(figsize=(7,4))
        self.ax = fig.add_subplot(111)    # facecolor='black'
        self.ax.set_xlabel('Fecha')
        self.ax.set_ylabel('Precio de Cierre ($)')
        
        self.mon = Label(frm1, text="Moneda").grid(column = 0, 
          row = 0, padx = 10, pady = 25)
        cmbBox = Combobox(frm1, values=Coins2, textvariable=self.Coin)
        self.lblPrecio = Label(frm1, text="", width=23   ,fg="green",font="Arial 15 bold")
        
        self.dtEntry1 = DateEntry(frm2,width=24,locale="es_PE")
        self.dtEntry1.set_date(startDate)
        self.dtEntry2 = DateEntry(frm2,width=24,locale="es_PE")  
        self.dtEntry2.set_date(endDate)        
        btnAsignar = Button(frm2,text="Obtener datos", width=13, command=self.update_tablas)
        
        self.notebook = Notebook(frm3,width=500)
        self.notebook.pack()
        frmNote1 = Frame(self.notebook,bg="white")
        frmNote2 = Frame(self.notebook,height=80)
        frmNote3 = Frame(self.notebook)
        #self.lblInfo = Label(frmNote1,text="",bg="white",justify="left")
        scrollbary = Scrollbar(frmNote2, orient="vertical")
        self.tabla = Treeview(frmNote2, columns=["Open","High","Close"],
                              yscrollcommand=scrollbary.set)
        scrollbary.config(command=self.tabla.yview)
        self.graph = FigureCanvasTkAgg(fig, master=frmNote3)
        self.graph.get_tk_widget().pack(side='top', fill='both', expand=True)
        
        
        
        
        cmbBox.grid(column=1,row=0,padx=7,pady=10)
        self.lblPrecio.grid(column=2,row=0)
        
        self.dtEntry1.grid(column=0,row=0,padx=11)
        self.dtEntry2.grid(column=1,row=0,padx=8)
        btnAsignar.grid(column=2, row=0,padx=11,pady=5)
        
        #self.notebook.add(frmNote1,text="Info")
        self.notebook.add(frmNote2,text="Tabla de Precios")
        self.notebook.add(frmNote3, text="Grafico Precio")
        #self.lblInfo.grid(padx=20,pady=15)
        self.tabla.pack(side="left",fill='both')
        scrollbary.pack(side="left", fill='y')
        
        self.tabla.heading("#0", text="Date")
        self.tabla.heading("#1", text="Open")
        self.tabla.heading("#2", text="High")
        #self.tabla.heading("#3", text="Low")
        self.tabla.heading("#3", text="Close")
                        
    
        self.tabla.column("#0", stretch=False, minwidth=80, width=150)
        self.tabla.column("#1", stretch=False, minwidth=60, width=110, anchor='e')
        self.tabla.column("#2", stretch=False, minwidth=60,  width=110, anchor='e')
        self.tabla.column("#3", stretch=False, minwidth=60,  width=110, anchor='e')
        
        print("Actualización de precio de la moneda cada 5 segundos en tiempo real")
        print("===================================================================")
        
                     
        
       
        
        self.master.after(10,self.start_animation)
        
    def start_animation(self):
        thr = threading.Thread(target=self.animate_prices, daemon=True)
        thr.start()
        
    def animate_prices(self):
        self.update_prices()
        
    def update_prices(self):
                
        coin_json = requests.get("https://api.coinbase.com/v2/prices/{}/spot".format(self.Coin.get()))       
        self.precio_viejo = self.precio
        self.precio = float(coin_json.json()["data"]["amount"])
        
        resta = self.precio-self.precio_viejo
#         print(resta)
        if self.precio_viejo!=0:
            
            print (f"Precio Actual: ${self.precio_viejo}")
            print(f"Precio hace 5 segundos: ${self.precio}")
            if resta<0:
                
                self.lblPrecio.config(text="{0:.4f} USD".format(self.precio),fg="green")
                print(f"Disminuyó la moneda en ${resta:,.2f}\n")
            elif resta>0:
                
                self.lblPrecio.config(text="{0:.4f} USD".format(self.precio),fg="red")
                print(f"Aumento la moneda en ${resta:,.2f}\n")
            else:
                
                self.lblPrecio.config(text="{0:.4f} USD".format(self.precio),fg="black")
                print("La moneda no ha subido ni bajado de precio\n")

        self.master.after(1000,self.update_prices)   
        

        
    
##
root = Tk()
Crypto = CryptoMonitor(root)
root.mainloop()

Actualización de precio de la moneda cada 5 segundos en tiempo real
Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3
Precio hace 5 segundos: $14068.3
La moneda no ha subido ni bajado de precio

Precio Actual: $14068.3